In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tdastro.astro_utils.passbands import PassbandGroup
from tdastro.astro_utils.pzflow_node import PZFlowNode
from tdastro.astro_utils.snia_utils import (
    DistModFromRedshift,
    HostmassX1Func,
    X0FromDistMod,
)
from tdastro.math_nodes.np_random import NumpyRandomFunc
from tdastro.obstable.opsim import OpSim
from tdastro.simulate import simulate_lightcurves
from tdastro.models.sncomso_models import SncosmoWrapperModel
from tdastro.models.snia_host import SNIaHost
from tdastro.utils.plotting import plot_lightcurves
from tdastro.math_nodes.ra_dec_sampler import ObsTableRADECSampler

from tdastro import _TDASTRO_BASE_DATA_DIR

In [2]:
# Load the OpSim data.
opsim_db = OpSim.from_db("/Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/opsim/baseline_v5.0.0_10yrs.db")

t_min, t_max = opsim_db.time_bounds()
print(f"Loaded OpSim with {len(opsim_db)} rows and times [{t_min}, {t_max}]")

# Load the passband data for the griz filters only.
# Use a (possibly older) cached version of the passbands to avoid downloading them.
passband_group = PassbandGroup.from_preset(
    preset="LSST",
    filters=["g", "r", "i", "z"],
    units="nm",
    trim_quantile=0.001,
    delta_wave=1,
)
print(f"Loaded Passbands: {passband_group}")

2025-09-08 00:33:57,792 - INFO - Loading passbands from preset LSST
2025-09-08 00:33:57,795 - INFO - Data file /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/u.dat already exists. Skipping download.
2025-09-08 00:33:57,795 - INFO - Loading passband from file: /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/u.dat
2025-09-08 00:33:57,802 - INFO - Data file /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/g.dat already exists. Skipping download.
2025-09-08 00:33:57,802 - INFO - Loading passband from file: /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/g.dat
2025-09-08 00:33:57,806 - INFO - Data file /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/r.dat already exists. Skipping download.
2025-09-08 00:33:57,807 - INFO - Loading passband from file: /Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/passbands/LSST/r.dat
2025-09-08 00:33:57,812

Loaded OpSim with 2048612 rows and times [60980.00162606769, 64632.272916088485]
Loaded Passbands: PassbandGroup containing 4 passbands: LSST_g, LSST_r, LSST_i, LSST_z


In [3]:
# Create a mask of matching filters.
filter_mask = passband_group.mask_by_filter(opsim_db["filter"])

# Filter the OpSim
opsim_db = opsim_db.filter_rows(filter_mask)
t_min, t_max = opsim_db.time_bounds()
print(f"Filtered OpSim to {len(opsim_db)} rows and times [{t_min}, {t_max}]")

Filtered OpSim to 1563870 rows and times [60980.00162606769, 64632.25331393084]


In [14]:
# Load the Flow model into a PZFlow node. This gives access to all of the outputs of the
# flow model as attributes of the PZFlowNode.
pz_node = PZFlowNode.from_file(
    _TDASTRO_BASE_DATA_DIR / "model_files" / "snia_hosts_test_pzflow.pkl",  # filename
    node_label="pznode",
)

# Create a model for the host of the SNIa. The attributes will be sampled via
# the PZFlowNode's model. So each host instantiation will have its own properties.
# Note: This requires the user to know the output names from the underlying flow model.
host = SNIaHost(
    ra=pz_node.RA_GAL,
    dec=pz_node.DEC_GAL,
    hostmass=pz_node.LOGMASS,
    redshift=NumpyRandomFunc("uniform", low=0.1, high=0.6),
    node_label="host",
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sidchaini/miniconda3/envs/tdastro/lib/python3.12/data/model_files/snia_hosts_test_pzflow.pkl'

In [11]:
model= SncosmoWrapperModel("salt2",
    # minimalist parameterization; sample redshift, phase t0, and amplitude
    redshift=NumpyRandomFunc("uniform", low=0.01, high=0.6),
    t0=NumpyRandomFunc("uniform", low=t_min, high=t_max),  # MJD range inside OpSim window
    x1=NumpyRandomFunc("normal", loc=0.0, scale=1.0),
    c=NumpyRandomFunc("normal", loc=0.0, scale=0.1),
    amplitude=NumpyRandomFunc("lognormal", mean=0.0, sigma=1.0),
    node_label="SNIa_salt2"
),

In [12]:
# Use an OpSim based sampler for position.
ra_dec_sampler = ObsTableRADECSampler(
    opsim_db,
    radius=3.0,  # degrees
    node_label="ra_dec_sampler",
)

time_sampler = NumpyRandomFunc("uniform", low=t_min, high=t_max, node_label="time_sampler")


In [13]:
lightcurves = simulate_lightcurves(model, 1_000, opsim_db, passband_group)

AttributeError: 'tuple' object has no attribute 'sample_parameters'